# Day 5

Tets input:

```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```

You basically need to create a series of mapping functions to go all the way from seed number to location. Report the lowest location. In the real input, numbers get huge.

In [1]:
test_input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [2]:
"".isspace()

False

In [7]:
# create a "mapper" class. This will be easier than using a dict. A mapper will take inputs from the statement and output numbers correctly.

class SeedMap:
    def __init__(self, name, container):
        self.name = name
        container = [[int(e) for e in line.split(" ")] for line in container]
        self.container = container

    def translate(self,num):
        for row in self.container:
            if num >= row[1] and num < row[1] + row[2]:
                return row[0] + (num - row[1])
        return num

def make_seed_maps(instructions_list):
    map_holder = []
    current_name = ""
    current_container = []
    for l in instructions_list[2:]:
        if not l:
            map_holder.append(SeedMap(current_name,current_container))
        elif l[0].isalpha():
            current_name = l
            current_container = []
        elif l[0].isnumeric():
            current_container.append(l)
    map_holder.append(SeedMap(current_name,current_container))
    return map_holder


def part1(instructions):
    # get seed nums
    # foor each group of instructions, make a mapper function
    instructions_list = instructions.split("\n")
    seeds = [int(s) for s in instructions_list[0].split(":")[1].strip().split(" ")]
    seed_maps_list = make_seed_maps(instructions_list)
    final_outputs = []
    for seed in seeds:
        s_num = seed
        for m in seed_maps_list:
            s_num = m.translate(s_num)
        final_outputs.append(s_num)
    return min(final_outputs)



In [8]:
part1(test_input) # translations are 82, 43, 86, 35 - 35 is answer 

35

In [9]:
with open("day5input.txt","r") as f:
    real_input = f.read()

In [10]:
part1(real_input)

457535844

## Part 2
same, but the seed number line is actually a list of two-ples. THe first is a start number, the second is a range, inclusive. still report the lowest num.

hmm... the second numbers are in the scale of millions (up to 100M). Might not be able to brute force this one. Gotta think a bit.

In [ ]:
def fix_seeds(seeds):
    all_seeds = []
    for i in range(0,len(seeds),2):
        s1 = seeds[i]
        s2

def part2(instructions):
    # get seed nums
    # foor each group of instructions, make a mapper function
    instructions_list = instructions.split("\n")
    seeds = [int(s) for s in instructions_list[0].split(":")[1].strip().split(" ")]
    seeds = fix_seeds(seeds)
    seed_maps_list = make_seed_maps(instructions_list)
    final_outputs = []
    for seed in seeds:
        s_num = seed
        for m in seed_maps_list:
            s_num = m.translate(s_num)
        final_outputs.append(s_num)
    return min(final_outputs)